<a href="https://colab.research.google.com/github/lidgarbar/Breast-Cancer-Model-Prediction/blob/main/ModelNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Breast Cancer Prediction Model 
> Authors: Juan Sánchez Moreno and Lidia García Barragán

## Setup and imports

In [1]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib pandas scikit-learn tqdm

  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


In [2]:
import os
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from tensorflow.keras.utils import load_img , img_to_array
import numpy as np
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline